This is just an exploration notebook to experiment with parso and help in development of new
features


In [2]:
import re
from pathlib import Path

import pandas as pd
import parso
from parso.python.tree import Function

In [3]:
path = "sample.py"
path = "../globality_black/tests/fixtures/ternary_input.txt"

In [26]:
file_contents = Path(path).read_text()
module = parso.parse(file_contents)

In [27]:
module

<Module: @4-12>

In [28]:
output = module.get_code()
type(output)
print(file_contents == output)
print(output[:1_000])

True
# in this file we cover lines using the ternary operator

# if very short, just leave it
variable = 1 if True else 0

def function_with_a_very_long_name_so_long_we_want_to_explode():
    return


# size 1 tuple non-exploded. Should stay the same
marc = function_with_a_very_long_name_so_long_we_want_to_explode() if True else False



In [29]:
x = module.children[0]
x

PythonNode(simple_stmt, [<ExprStmt: # in this file we cover lines using the ternary operator  # if very short, just leave it variable = 1 if True else 0@4,0>, <Newline: '\n'>])

In [30]:
x = x.children[0]
x

<ExprStmt: # in this file we cover lines using the ternary operator  # if very short, just leave it variable = 1 if True else 0@4,0>

In [33]:
x.children[2].__dir__()
x.children[2].end_pos

(4, 27)

In [ ]:
x.prefix

In [ ]:
x.get_first_leaf()

In [ ]:
x.children[0].children[0].prefix

In [ ]:
y = module.get_leaf_for_position(x.start_pos)

In [ ]:
y.prefix

In [ ]:
x.get_first_leaf()

In [ ]:
func = next(module.iter_funcdefs())
func

In [ ]:
func.children[2].children[-1].get_first_leaf()

In [ ]:
def detect_magic_comma(function: Function):
    # get last param in signature. get its children and
    # check if the last one equals ","
    if len(function.get_params()) > 0:
        return function.get_params()[-1].children[-1] == ","
    return False

## detect magic comma for functions

In [ ]:
print(module.children[0].type == "funcdef")

In [ ]:
# go through every function (class methods TODO)
f = module.children[1]
print(f.get_code())

In [ ]:
params = f.get_params()
params

In [ ]:
params[-1].children

In [ ]:
detect_magic_comma(f)

## detect magic comma for class methods

In [ ]:
c = module.children[3]
c

In [ ]:
for function in c.iter_funcdefs():
    print(f"{function.name.value}: {detect_magic_comma(function)}")

In [ ]:
cf = list(c.iter_funcdefs())[1]
list(cf.iter_funcdefs())

In [ ]:
def get_indent_first_param(function):
    pos = function.get_params()[0].start_pos
    prefix = function.get_leaf_for_position(pos).prefix
    return get_indent_from_prefix(prefix)

In [ ]:
def get_indent_from_prefix(prefix):
    if prefix:
        try:
            return re.findall("( *)$", prefix)[0]
        except Exception:
            breakpoint()
    else:
        return prefix

In [ ]:
def correct_prefix_if_magic_comma(function):

    if detect_magic_comma(function):
        params = function.get_params()
        prefix = function.get_first_leaf().prefix
        base_indent = get_indent_from_prefix(prefix)
        for param in params:

            leaf = param.get_first_leaf()
            leaf.prefix = "\n" + base_indent + " " * 4

        # get right parentheses
        leaf = function.children[2].children[-1].get_first_leaf()
        leaf.prefix = "\n" + base_indent

    for nested_function in function.iter_funcdefs():
        correct_prefix_if_magic_comma(nested_function)

In [ ]:
for c in module.children:
    print(c)

In [ ]:
module.children[1].type

## Experiment with function calls

In [ ]:
print(module.get_code())

In [ ]:
class TypeFinder:
    def __init__(self, types_to_find):
        self.types_to_find = types_to_find

    def __call__(self, node):
        if node.type in self.types_to_find:
            yield node
        if hasattr(node, "children"):
            for child in node.children:
                yield from self(child)

In [ ]:
arglist = module.children[2].children[0].children[1].children[1]
arglist

In [ ]:
# finder = TypeFinder(["funcdef", "arglist"])
finder = TypeFinder(["parameters", "arglist"])
for elem in finder(module):
    print(elem)
    break

In [ ]:
elem.get_params()[-1]

In [ ]:
elem.children[-2]

In [ ]:
elem.get_params()[-1].children[-1]

In [ ]:
print(elem.get_code())

In [ ]:
elem.children

## Dotted chains

In [4]:
path = "../globality_black/tests/fixtures/dotted_chains_input.txt"

In [206]:
file_contents = Path(path).read_text()
module = parso.parse(file_contents)

In [200]:
output = module.get_code().split("\n")
print("\n".join(output[:8]))

import pandas as pd


def foo():
    batch = (
        batch
        .sample(
            frac=param1,


In [207]:
block1 = module.children[1].children[4].children[1].children[0]
print(block1.get_code())
block1

    batch = (
        batch(a)(b)
        .sample(
            frac=param1,
            rs=param2,
        )
        .reset_index(drop1=True)(drop2=True)
        .reset_index(drop=True).reset_index(drop=True)
    )


<ExprStmt: batch = (         batch(a)(b)         .sample(             frac=param1,             rs=param2,         )         .reset_index(drop1=True)(drop2=True)         .reset_index(drop=True).reset_index(drop=True)     )@5,4>

In [85]:
block2 = module.children[2].children[0]
print(block2.get_code())
block2


LABELS = set(
    df[df.labels.apply(len) > 0]
    .flag.apply(curate)
    .apply(normalize)
    .unique()
)


<ExprStmt: LABELS = set(     df[df.labels.apply(len) > 0]     .flag.apply(curate)     .apply(normalize)     .unique() )@10,0>

In [86]:
block3 = module.children[3]
print(block3.get_code())
block3


return (
    df_field[COLUMNS_PER_FIELD[name]]
    .dropna(subset=["column"])
    .reset_index(drop=True)
    .assign(mapped_type=MAP_DICT[name])
)


<ReturnStmt: return (     df_field[COLUMNS_PER_FIELD[name]]     .dropna(subset=["column"])     .reset_index(drop=True)     .assign(mapped_type=MAP_DICT[name]) )@17,0>

In [208]:
block1.children

[<Name: batch@5,4>,
 <Operator: =>,
 PythonNode(atom, [<Operator: (>, PythonNode(atom_expr, [<Name: batch@6,8>, PythonNode(trailer, [<Operator: (>, <Name: a@6,14>, <Operator: )>]), PythonNode(trailer, [<Operator: (>, <Name: b@6,17>, <Operator: )>]), PythonNode(trailer, [<Operator: .>, <Name: sample@7,9>]), PythonNode(trailer, [<Operator: (>, PythonNode(arglist, [PythonNode(argument, [<Name: frac@8,12>, <Operator: =>, <Name: param1@8,17>]), <Operator: ,>, PythonNode(argument, [<Name: rs@9,12>, <Operator: =>, <Name: param2@9,15>]), <Operator: ,>]), <Operator: )>]), PythonNode(trailer, [<Operator: .>, <Name: reset_index@11,9>]), PythonNode(trailer, [<Operator: (>, PythonNode(argument, [<Name: drop1@11,21>, <Operator: =>, <Keyword: True>]), <Operator: )>]), PythonNode(trailer, [<Operator: (>, PythonNode(argument, [<Name: drop2@11,33>, <Operator: =>, <Keyword: True>]), <Operator: )>]), PythonNode(trailer, [<Operator: .>, <Name: reset_index@12,9>]), PythonNode(trailer, [<Operator: (>, Python

In [88]:
block2.children[2].children

[<Name: set@10,9>,
 PythonNode(trailer, [<Operator: (>, PythonNode(atom_expr, [<Name: df@11,4>, PythonNode(trailer, [<Operator: [>, PythonNode(comparison, [PythonNode(atom_expr, [<Name: df@11,7>, PythonNode(trailer, [<Operator: .>, <Name: labels@11,10>]), PythonNode(trailer, [<Operator: .>, <Name: apply@11,17>]), PythonNode(trailer, [<Operator: (>, <Name: len@11,23>, <Operator: )>])]), <Operator: >>, <Number: 0>]), <Operator: ]>]), PythonNode(trailer, [<Operator: .>, <Name: flag@12,5>]), PythonNode(trailer, [<Operator: .>, <Name: apply@12,10>]), PythonNode(trailer, [<Operator: (>, <Name: curate@12,16>, <Operator: )>]), PythonNode(trailer, [<Operator: .>, <Name: apply@13,5>]), PythonNode(trailer, [<Operator: (>, <Name: normalize@13,11>, <Operator: )>]), PythonNode(trailer, [<Operator: .>, <Name: unique@14,5>]), PythonNode(trailer, [<Operator: (>, <Operator: )>])]), <Operator: )>])]

In [89]:
block3.children[1]  # .children[1].children[1]

PythonNode(atom, [<Operator: (>, PythonNode(atom_expr, [<Name: df_field@18,4>, PythonNode(trailer, [<Operator: [>, PythonNode(atom_expr, [<Name: COLUMNS_PER_FIELD@18,13>, PythonNode(trailer, [<Operator: [>, <Name: name@18,31>, <Operator: ]>])]), <Operator: ]>]), PythonNode(trailer, [<Operator: .>, <Name: dropna@19,5>]), PythonNode(trailer, [<Operator: (>, PythonNode(argument, [<Name: subset@19,12>, <Operator: =>, PythonNode(atom, [<Operator: [>, <String: "column">, <Operator: ]>])]), <Operator: )>]), PythonNode(trailer, [<Operator: .>, <Name: reset_index@20,5>]), PythonNode(trailer, [<Operator: (>, PythonNode(argument, [<Name: drop@20,17>, <Operator: =>, <Keyword: True>]), <Operator: )>]), PythonNode(trailer, [<Operator: .>, <Name: assign@21,5>]), PythonNode(trailer, [<Operator: (>, PythonNode(argument, [<Name: mapped_type@21,12>, <Operator: =>, PythonNode(atom_expr, [<Name: MAP_DICT@21,24>, PythonNode(trailer, [<Operator: [>, <Name: name@21,33>, <Operator: ]>])])]), <Operator: )>])]),

atom_expr is the types to inspect here

### is_dotted_chain


ae1 = block1.children[2].children[1]
print(ae1.get_code())
print(ae1.type)
pd.DataFrame(dict(elem=ae1.children)).assign(
    code=lambda df: df.elem.apply(lambda e: e.get_code()),
    _type=lambda df: df.elem.apply(lambda e: type(e).__name__),
    a_type=lambda df: df.elem.apply(lambda e: e.type),
    first_leaf_prefix=lambda df: df.elem.apply(lambda e: "'" + e.get_first_leaf().prefix) + "'",
)
pd.Series(ae1).apply(lambda e: e.type)

In [216]:
def is_dotted_chain(atom_expr):
    """
    Assert whether the given atom_expr is a dotted chain

    Examples:

    batch(a)(b)
    .sample(
        frac=param1,
        rs=param2,
    )
    .reset_index(drop1=True)(drop2=True)
    .reset_index(drop=True).reset_index(drop=True)
    --> True

    batch
    (batch)
    --> False

    batch,
    batch,
    --> False

    batch
    .batch
    --> True

    batch
    .batch(
        x=4,
    )
    --> True

    """
    # print(atom_expr.get_code())
    # print(atom_expr)
    children = atom_expr.children

    # first node should start with \n + tabs (sapces multiple of 4)
    prefix_child1 = children[0].get_first_leaf().prefix
    if not re.match(r"\n(?:\s{4})+", prefix_child1):
        return False

    # more than one child needed to be a dotted chain
    if len(children) < 2:
        return False

    # all children after the first one that start with \n + tabs should have a . afterwards
    # otherwise it's not a dotted chain
    for child in children[1:]:
        code = child.get_code()
        if re.match(r"\n(?:\s{4})+", code):
            if not code.strip().startswith("."):
                return False

    return True


def read_and_turn_to_ae(code):
    m = parso.parse("labels = (" + code + "\n)")
    #     breakpoint()
    return m.children[0].children[2].children[1]


print(
    is_dotted_chain(ae1),
    is_dotted_chain(read_and_turn_to_ae("\n    batch\n    (batch)")),
    is_dotted_chain(read_and_turn_to_ae("\n    batch,\n    batch,")),
    is_dotted_chain(read_and_turn_to_ae("\n    batch\n    .batch")),
    is_dotted_chain(read_and_turn_to_ae("\n    batch\n    .batch(\n        x=4,\n    )")),
)


        batch(a)(b)
        .sample(
            frac=param1,
            rs=param2,
        )
        .reset_index(drop1=True)(drop2=True)
        .reset_index(drop=True).reset_index(drop=True)
atom_expr


,elem,code,_type,a_type,first_leaf_prefix
0,"<Name: batch@6,8>",\n batch,Name,name,'\n '
1,"PythonNode(trailer, [<Operator: (>, <Name: a@6...",(a),PythonNode,trailer,''
2,"PythonNode(trailer, [<Operator: (>, <Name: b@6...",(b),PythonNode,trailer,''
3,"PythonNode(trailer, [<Operator: .>, <Name: sam...",\n .sample,PythonNode,trailer,'\n '
4,"PythonNode(trailer, [<Operator: (>, PythonNode...","(\n frac=param1,\n rs=pa...",PythonNode,trailer,''
5,"PythonNode(trailer, [<Operator: .>, <Name: res...",\n .reset_index,PythonNode,trailer,'\n '
6,"PythonNode(trailer, [<Operator: (>, PythonNode...",(drop1=True),PythonNode,trailer,''
7,"PythonNode(trailer, [<Operator: (>, PythonNode...",(drop2=True),PythonNode,trailer,''
8,"PythonNode(trailer, [<Operator: .>, <Name: res...",\n .reset_index,PythonNode,trailer,'\n '
9,"PythonNode(trailer, [<Operator: (>, PythonNode...",(drop=True),PythonNode,trailer,''


In [246]:
type(module.children[0])

parso.python.tree.PythonNode

In [214]:
re.match(r"\n(?:\s{4})+", ae1.children[0].prefix)

<re.Match object; span=(0, 9), match='\n        '>

In [174]:
ae1.children[0].get_code()

'\n        batch'

In [220]:
ae1.children[3].get_code().strip()

'.sample'

In [118]:
ae1[1].children[0]

<Operator: .>